In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Embedding, Dense, LSTM, Dropout, TimeDistributed, Lambda, Flatten, GlobalAveragePooling1D
from keras import layers
import keras.backend as K
import tensorflow as tf
from keras import optimizers
#import random
import math
#import tf.contrib.distributions.NormalWithSoftplusScale as NORM


import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

import theano

def plot(label,prediction,vi, num_plot,k,e): # 1,192  1,192  1,192
    x = np.arange(192)
    f = plt.figure()
    base = num_plot*100+10
    for i in range(num_plot):
        label_temp = label[i].reshape([window_length,]) * vi[i]
        pred_temp = prediction[i].reshape([window_length,]) * vi[i]
        plt.subplot(base+i+1)
        plt.plot(x,label_temp, color='b')
        plt.plot(x,pred_temp, color='r')        
        plt.axvline(168, color='k')
    #plt.pause(5)
#    plot.show()
    print('saving...')
    f.savefig(str(e)+'thEpoch'+str(k)+"thBatch.png")
    plt.close()



def log_gaussian(x, mean, std):
    #dist = tf.contrib.distributions.NormalWithSoftplusScale(mean,std)
    dist = tf.contrib.distributions.Normal(mean,std)
    likelihood = dist.log_prob(x)
    return likelihood

def sum_log_likelihood(y_true, para_pred):
    print('\n==in custom loss===')
    print("y_true.shape: ", y_true.shape)
    print("para_pred.shape: ", para_pred.shape)
    mean = para_pred[:,:,0]
    std = (para_pred[:,:,1])
    print("mean.shape: ", mean.shape)
    print('std.shape: ', std.shape)
    likelihood = log_gaussian(y_true[:,:,0], mean, std)
    print("likelihood.shape: ",likelihood.shape)
    print('==end of custom loss===')
    result =  K.mean(likelihood)
    return -result


data = np.load('reframed-data-all.npy')
#data = data[0:1000, :, :]
v_i = np.load('vi-g-all.npy')
#v_i = v_i[0:1000, :]
print("data.shape: ", data.shape)
print("v_i.shape: ", v_i.shape)


# get the dimension
input_window_length = 168
output_window_length = 24
window_length = input_window_length + output_window_length
n_features = 4
n_dims = 370
input_embed_dim = 370
output_embed_dim = 20
n_samples = data.shape[0]
N = ((int(n_samples * 0.95)) // 64) * 64 # number of samples in train data

aux_in = Input(shape=(None, ), name='aux_input', dtype='int32')

aux_in_full = Lambda(K.one_hot, arguments={'num_classes': n_dims}, output_shape=(None, n_dims))(aux_in)
x = Dense(20, activation='sigmoid')(aux_in_full)

main_in = Input(shape=(None, n_features, ), name="main_input")
input1 = layers.concatenate([main_in, x])

lstm_out1 = LSTM(40, return_sequences = True)(input1)
drop_out1 = Dropout(0.2)(lstm_out1)
lstm_out2 = LSTM(40,  return_sequences = True)(drop_out1)
drop_out2 = Dropout(0.2)(lstm_out2)
lstm_out3 = LSTM(40,  return_sequences = True)(lstm_out2)
drop_out3 = Dropout(0.2)(lstm_out3)


mean_for_each = TimeDistributed(Dense(1))(drop_out3)
std_for_each = TimeDistributed(Dense(1, activation='softplus'))(lstm_out3)
out_for_each = layers.concatenate([mean_for_each,std_for_each],axis = 2)

In [1]:
model = Model(inputs=[aux_in,main_in], outputs=[mean_for_each, std_for_each])
adam = optimizers.Adam(lr=0.01)
model.compile(loss="mean_squared_logarithmic_error", optimizer=adam)
#model.compile(loss=sum_log_likelihood, optimizer=adam)
print(model.summary())

train_main_input = data[:N,:, 0:4] # ground truth and covariates
train_aux_input =  np.array(data[:N,:,4], dtype='int32') # the one-hot position
#train_aux_input = (np.arange(n_dims) == train_aux_input[...,None]-1).astype(np.int32, copy=False)
train_y = data[:N,:,5].reshape(-1, window_length, 1)

print(train_main_input.shape, train_aux_input.shape)


train_y.shape

test_main_input = data[N:,:,0:4] # ground truth and covariates
test_aux_input = np.array(data[N:,:,4], dtype='int32') # the one-hot position
test_vi = v_i[N:, :]
rewritten_input = np.copy(test_main_input)
batch_size = 32
n_batch = (n_samples - N) // batch_size

nd = np.zeros(n_batch)
rmse = np.zeros(n_batch)

n_epoches = 1


def rmse_metrics(y_true, mean, vi):
    y_true = y_true * vi
    mean = mean * vi
    denom = np.mean(np.absolute(y_true))
    if (denom == 0.0):
        denom = -1.0
    return math.sqrt(np.mean(np.square(mean-y_true)))/denom

def nd_metrics(y_true, mean, vi):
    y_true = y_true * vi
    mean = mean * vi
    denom = np.sum(np.absolute(y_true))
    if (denom == 0.0):
        denom = -1.0
    return np.sum(np.absolute(mean-y_true))/denom

Using TensorFlow backend.


In [27]:
for e in range(n_epoches):
    print('epoch: ',e+1, '/', n_epoches)
    model.fit([train_aux_input,train_main_input], [train_y] , epochs=1, batch_size=32,verbose=1, shuffle=True)
    model.save_weights(str(e)+'th_Epoch_weights.h5')
    for i in range(n_batch):
        batch_range = range(i*batch_size, (i+1)*batch_size)
        for j in range(output_window_length): # j = [0.23], input_window_length+j = [168,191]
            #=========== index, input, dimension check ====================
            #print('\n========= now predicting the index (start from 0): ',input_window_length+j, '=========')
            #print('which means that the current input is: [0,' + str(input_window_length+j-1) + ']')
            #print('aka. ":'+str(input_window_length+j)+'"')
            main_input = rewritten_input[batch_range,:input_window_length + j,0:4]
            #print('so the main input for this round of prediction is:\n', main_input)
            #print('shape of the main input for this round of prediction is: ', main_input.shape) 
            # from (64,168,4) to (64,191,4)
            aux_input = test_aux_input[batch_range,:input_window_length + j]
            #print('shape of the auxiliary input for this round of prediction is: ', aux_input.shape) 
            # from (64,168) to (64,191)
            #=========== make the prediction ==============================
            pred_result = model.predict_on_batch([aux_input, main_input])
            #========== get prediction for next sequence ==================
            rewritten_input[batch_range, input_window_length + j, 0] = pred_result[:, -1 ,0]
            #========== draw the prediction ===============================
        
        nd[i] = nd_metrics(test_main_input[batch_range, input_window_length:, 0], rewritten_input[batch_range, input_window_length:, 0], test_vi[batch_range])
        rmse[i] = rmse_metrics(test_main_input[batch_range, input_window_length:, 0], rewritten_input[batch_range, input_window_length:, 0], test_vi[batch_range])
        if (nd[i] <= 0.09 or rmse[i] <= 0.4 or (nd[i]<0.11 and rmse[i]<0.5)):
            plot(test_main_input[i*batch_size:i*batch_size+8,:,0], rewritten_input[i*batch_size:i*batch_size+8,:,0], test_vi[i*batch_size:i*batch_size+8],8,i,e)
            print(nd[i], rmse[i])
    print('nd average: ', np.mean(nd))
    print('rmse average: ', np.mean(rmse))
    
    print('nd min: ', np.min(nd))
    print('rmse min: ', np.min(rmse))
    
    print('nd max: ', np.max(nd))
    print('rmse max: ', np.max(rmse))
    
    np.save(str(e)+'thEpochOfND.npy', nd)
    np.save(str(e)+'thEpochOfRMSE.npy', rmse)


('epoch: ', 1, '/', 1)
Epoch 1/1
412480/412480 [==============================] - 7960s 19ms/step - loss: 0.0034
saving...
(0.06406789346662124, 0.19568581893089138)
saving...
(0.0824951705952719, 0.18316715325884653)
saving...
(0.08342865966546677, 0.21778168534949657)
saving...
(0.06932534503023759, 0.3541152743677562)
saving...
(0.07495595686479152, 0.45123543186677034)
saving...
(0.06160737065784354, 0.10913310211896918)
saving...
(0.07278088990696054, 0.1431974689251798)
saving...
(0.09479884178444298, 0.22656972947746362)
saving...
(0.06788978636510778, 0.15024976413355762)
saving...
(0.056289661469732195, 0.30056702045673866)
saving...
(0.06227650998758103, 0.22041027265573288)
saving...
(0.05392237447492655, 0.1805338585182529)
saving...
(0.08834967889838229, 0.22266016650218978)
saving...
(0.08758844711215365, 0.22209681214588844)
saving...
(0.08622071219627975, 0.2008304749283855)
saving...
(0.04361565155526009, 0.16569935132881428)
saving...
(0.04315775903466117, 0.146835821

saving...
(0.054776694806338316, 0.09298858831734774)
saving...
(0.084066969670894, 0.19768696752931644)
saving...
(0.06955780545341655, 0.23923505810431472)
saving...
(0.06988611219174534, 0.4328012708751327)
saving...
(0.10178731450698691, 0.4247008409160271)
saving...
(0.056775324784037996, 0.15410295806751526)
saving...
(0.07786825862382458, 0.19403742870257618)
saving...
(0.07602393212293469, 0.1459767039584732)
saving...
(0.07512680500178318, 0.2635505844560569)
saving...
(0.0872747656996751, 0.20545616497796781)
saving...
(0.11608126902581725, 0.26875760630835566)
saving...
(0.10921258300492068, 0.24076867218558448)
saving...
(0.08283226041098785, 0.2096597285831406)
saving...
(0.0683422953501187, 0.29557499219728217)
saving...
(0.07905734161874323, 0.23368788051427378)
saving...
(0.06862659439755496, 0.13109273508711478)
saving...
(0.07927810794678178, 0.29427499096033893)
saving...
(0.07024703624454108, 0.15627173917431467)
saving...
(0.06668783626749042, 0.14201070877208988)


saving...
(0.07397976692010143, 0.163020023276397)
saving...
(0.07093125810724622, 0.36088688008877284)
saving...
(0.1373207688575817, 0.3150473483818157)
saving...
(0.10104248171727807, 0.3844060401261116)
saving...
(0.06298894104050427, 0.13773676653009387)
saving...
(0.07566203611673306, 0.41597026839462203)
saving...
(0.06956643589385723, 0.15604425855107235)
saving...
(0.07060009962103903, 0.2712273410392617)
saving...
(0.08090167408765994, 0.21571381100650486)
saving...
(0.09502619406941604, 0.2629196140490642)
saving...
(0.0889257905393459, 0.23021111371897887)
saving...
(0.13424562412243077, 0.30634776951770265)
saving...
(0.06959700337964049, 0.14998980416233493)
saving...
(0.0701008968444914, 0.22223822375941632)
saving...
(0.0862904008317897, 0.23327422810282472)
saving...
(0.12220254888684115, 0.33556024695236203)
saving...
(0.09474002474787087, 0.2447208176614184)
saving...
(0.08172389669352517, 0.1796107108221837)
saving...
(0.11481993751251493, 0.25975717343580224)
savin

saving...
(0.08967035482994681, 0.22160806259525298)
saving...
(0.056000927568984685, 0.2584861473671135)
saving...
(0.1035096103466351, 0.2633790288697467)
saving...
(0.0798165244182026, 0.16555770538798747)
saving...
(0.12296851166808306, 0.27444374776585145)
saving...
(0.10874337749984729, 0.22343811430809862)
saving...
(0.0907702210271072, 0.27741022860868453)
saving...
(0.08547574673669892, 0.22255218167209115)
saving...
(0.054556565105560374, 0.16999029870554977)
saving...
(0.085022177130965, 0.3748168465907071)
saving...
(0.06684560823997077, 0.19283598769235527)
saving...
(0.0793552969368343, 0.19844631239805682)
saving...
(0.07283120658229318, 0.16682817175286616)
saving...
(0.07200513074345005, 0.2066077801473885)
saving...
(0.09724945690097864, 0.24992455677119343)
saving...
(0.0817921602233286, 0.2544376005608513)
saving...
(0.08711673676907757, 0.4319684367740514)
saving...
(0.07859719473166553, 0.13153312379262877)
saving...
(0.08664436708117697, 0.3647838762759968)
savin

KeyboardInterrupt: 